# This notebook presents some approaches to cluster the dataset of the Netflix Challenge #

This notebook is written in python. Eventually, the kernels have to be rewritten in julia. 
First we load the sparse Matrix containing the date, which was preprocessed by Thomas Camminady.


In [1]:
# unzip the files
from zipfile import ZipFile

# Create a ZipFile Object and load sample.zip in it
with ZipFile('NPY.zip', 'r') as zipObj:
   # Extract all the contents of zip file in current directory
   zipObj.extractall()

In [2]:
import numpy as np
# Load the sparse matrix

I = np.load('NPY/I.npy',allow_pickle=True) #Movie ID
J = np.load('NPY/J.npy',allow_pickle=True) #User ID
V = np.load('NPY/V.npy',allow_pickle=True) #Rating of user j for movie i.

# Show some stats
print("Non-zero entries of the rating matrix: ")
print(I.size)
print("Number of unique movies in the training set, i.e. columns of the matrix:")
print(np.unique(I).size)
print("Number of unique users in the training set, i.e. rows of the matrix:")
print(np.unique(J).size)
print("The highest user id is:")
print(np.amax(J))

Non-zero entries of the rating matrix: 
100480507
Number of unique movies in the training set, i.e. columns of the matrix:
17770
Number of unique users in the training set, i.e. rows of the matrix:
480189
The highest user id is:
2649429


In [3]:
# generate sparse arrays that are more suitable for our needs
user_count  = np.amax(J)+1
rating_count = I.size
# Generate an array with index = user_id.
user_ratings = np.empty((user_count,),dtype=object)

# Each user has a list of touples (movie_id, rating) representing his rated movies.
for idx in range(0,user_count):
    user_movies = list()
    user_ratings[idx] = user_movies

#assign to each user his list of rated films.
for rating_idx in range(0,rating_count):
    movie_tuple = (I[rating_idx],V[rating_idx])
    user_id= J[rating_idx]  
    user_ratings[user_id].append(movie_tuple)


In [4]:
# sort the rated movies of each user by id
for ratings in user_ratings: 
    ratings.sort(key=lambda tup: tup[0])


## Analysing the problem ##
The task at hand is to predict, how a given user rates a given movie. To fulfill this task, we have a (preprocessed) training dataset, that contains a sparse matrix S with entries S[i,j] = v, interpreted as movie i was ranked by user j with v stars. Additionally to v, we also store y, m, and d which represent the year, month, and day on which the rating was given.

First, we try to solve the problem looking only at the ratings of the users and ignoring all other data, i.e. the date of the rating and the release date of the movie. 

## Idea 1: Clustering ##
One general Idea to solve this problem is to cluster the users, i.e. group users with shared interests. Once we can assign a user to the user group with which he has the most common interests, we can supplement the missing information about that user using the information from the group.
The interest of the group can be modelled by creating the mean of the interests of its members. In our context, this means, that we assume, that the group rates a movie by the mean of all available ratings of its members. Then we simply assume, that the new user gives the film the same rating as the average of the group he was assigned to.

Whereas it is rather simple to perform the second task of our idea, i.e. assign the mean rating to the new user, given that we know his group, it is not trivial to cluster the users in a meaningful manner. 

- The first question, we have to ask, is: What is the space, in which we cluster the users? 
    The first thing that comes into mind is that the clustering space is the space of movies, i.e. each movie represents one dimension of the clustering space. 
    In our context, this means that we have 17770 movies, i.e. 17770 dimensions of the clustering space. 
    We have 480189 users, i.e. 480189 data points in this clustering space, that need to be clustered. 
    
- The second quastion: How many user groups, i.e. clusters, do we need? 
    This question is hard to answer a priori. Furthermore, many clustering algorithms need this information before the execution. 
    


In the following, we present some clustering algorithms to perform this task.

### 1a) K-means clustering ###

(Text copied from [here](https://en.wikipedia.org/wiki/K-means_clustering) )

Training: 

Given an initial set of $k$ means $m_1,\dots,m_k$, the algorithm proceeds by alternating between two steps:

*Assignment step*: Assign each observation to the cluster with the nearest mean: that with the least squared  Euclidean distance. (Mathematically, this means partitioning the observations according to the Voronoi diagram generated by the means.)
 -  $S_i^{(t)} = \big \{ x_p : \big \| x_p - m^{(t)}_i \big \|^2 \le \big \| x_p - m^{(t)}_j \big \|^2 \ \forall j, 1 \le j \le k \big\}$,
 -  where each $x_p$ is assigned to exactly one $S^{(t)}$, even if it could be assigned to two or more of them.
 
*Update step*: Recalculate means (centroids) for observations assigned to each cluster.
 - $m^{(t+1)}_i = \frac{1}{\left|S^{(t)}_i\right|} \sum_{x_j \in S^{(t)}_i} x_j $

The algorithm has converged when the assignments no longer change. The algorithm does not guarantee to find the optimum.

Classification:
A given user is assigned to the group with smallest distance. Then the predicted rating of a movie is given by the mean of the ratings of the group.

The algorithm is often presented as assigning objects to the nearest cluster by distance. Using a different distance function other than (squared) Euclidean distance may stop the algorithm from converging. Various modifications of $k$-means such as spherical $k$-means and K-medoids have been proposed to allow using other distance measures.

#### Problems with K-means: #### 
Some problems arise when we naively try to apply $k$-means clustering
- What is the location of a user (datapoint) in a movie (dimension), that he has not rated? How can we compute a distance, when not every user has rated every movie?

- [Curse of dimensionality](https://en.wikipedia.org/wiki/Curse_of_dimensionality): In high dimensional datasets, a dataset almost always appears to be sparse. Therefore, the datapoints are very likely to have large distances between them and we have problems to assign them to a cluster.
   
   In a mathematical context, this is paraphrased to the following: A $k$-means algorithm tries to minimize the variation of datapoints of a cluster by reassigning them. If the clustering space is too high dimensional, chances are high, that a given dataset has not enough statistical significance to be suited for such a procedure.
   
   

In [5]:
# k-means implementation

# Step 1: Create a better accessible sparse matrix pattern:
#         Each user is assigned to a list of touples (movie_id, rating), of movies that he has rated.



### 1b) $k$- nearest neighbors clustering ###

[$k$-nn](https://en.wikipedia.org/wiki/K-nearest_neighbors_algorithm) clustering is another method to classify a datapoints, i.e. predict the movie rating of a user. 
In contrast to the $k$- means clustering method, $k$-nn clustering does not assign the user to a group of similarily minded users. Therefore we do not have to think about a number of different user-groups before we start the algorithm. 

The idea is to look at the $k$ nearest neighbors of a given user. Depending on their ratings of the movie at hand, we determine the predicted rating for the given user and the movie at hand. 

There is no training phase for the $k$-nn algorithm in the sense of the $k$-means algorithm above. 

However, he problems with $k$-means persist in $k$-nn.


#### Solution Suggestions ####

1. Do not use Euklidean Distance, but instead a Similarity Coefficient, e.g. Pearson Correlation, computed from the correlation of the ratings of movies, which are rated by a pair of users.

2. Reduce Dimensionality by SVD or PCA


ad 1) [Pearson Correlation Coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) when applied to a sample is commonly represented by $r_{xy}$ and may be referred to as the ''sample correlation coefficient'' or the ''sample Pearson correlation coefficient''. We can obtain a formula for $r_{xy}$ by substituting estimates of the covariances and variances based on a statistical sample into the formula above. Given paired data $\left\{ (x_1,y_1),\ldots,(x_n,y_n) \right\}$ consisting of $n$ pairs, $r_{xy}$ is defined as:

$
r_{xy} =\frac{\sum\limits_{i \in I_{xy}}(r_{x,i}-\bar{r_x})(r_{y,i}-\bar{r_y})}{\sqrt{\sum\limits_{i \in I_{xy}}(r_{x,i}-\bar{r_x})^2}\sqrt{\sum\limits_{i \in I_{xy}}(r_{y,i}-\bar{r_y})^2}}
$

where:
 - I is the index set of available samples. In our case, these are all movies that are rated by user x and y
 - $n = \vert I\vert$ is sample size
 - $r_{x,i}, r_{y,i}$ are the individual sample points indexed with $i$
 - $\bar{r_x}=\frac{1}{n}\sum_{i=1}^n r_{x,i}$ (the sample mean); and analogously for $\bar{r_y}$
 

In comparison, an Euklidean distance between two users is defined as
 
 $
 d_{x,y} = \sqrt{\sum\limits_{i=1}^N (r_{x,i}-{r_{y,i}})^2}
 $
 
 where $N$ is the number of movies, i.e. the dimensionality of the clustering space. Here we need to use each movie, i.e. dimension, even if it is not rated by the user.
 
 
Remarks:
    - Are user pairs with a low number of mutual ratings problematic? (I think not)


## 1b) $k$-nn revisited ##

Using the Pearson Corellation Coefficient, we get the following modified $k$-nn algorithm to predict the rating of user $x$ for movie $m$, $R_{x,m}$.
Let $X$ be the set of users, $M$ be the set of movies.

0. Set the number of considered neighbors $k$, choose a movie id $m$ and a user-id $x$.
1. for $y\in X, y\not=x$:

    a) compute $I_{x,y}=\{n\in M : n \text{ is rated by } x \text{ and } y \}$
    
    b) compute $r_{xy}$ and store it.
    
2. Find the $y_1,\dots,y_k$ with the highest values of $r_{x,y}$. 
3. $R_{x,m}= \frac{1}{k}\sum_{i=1}^k R_{y_i,m}$


In [41]:
# helper function intersection
def custom_intersection(list1, list2):
    idx_1 = 0;
    idx_2 = 0;
    res_list = list(); # list of triples [movie_id, rating_x, rating_y]
    while idx_1 < len(list1) and idx_2 < len(list2):
        if list1[idx_1][0] == list2[idx_2][0]:
            res_list.append( [list1[idx_1][1], list2[idx_2][1] ])
            idx_1 += 1 
            idx_2 += 1
        elif list1[idx_1][0] > list2[idx_2][0]:
            idx_2 += 1
        else:
            idx_1 += 1
            
    return res_list

# helper function getMovieRating (assumes, that user_ratings exist)
def getRating(id_user, id_movie):

    user = user_ratings[id_user];

    rating = -1 # token for non rated movies
    
    for key, value in user:
            if key == id_movie:
                rating = value;
    
    return rating #if user has not rated, return -1

# k_nn implementation, that outputs the k nearest neighbors of user_id
# assumes that the user_ratings sparse matrix is already in memory

def k_nn_memorized(user_id, k, users_mat, target_user, minimum_intersection = 5):
    
    '''
    user_id: user id of target_user
    user_mat: sparse matrix of all training users with their movies (array of list of tuples)
    target_user: user with id user_id
    minimum_intersectioN: target_user and its current neighbour candidate must have rated the same <minimum_intersection> movies.
    
    return: nearest neighbors: list of touples (user_id, pearson_corr with target user)
    '''
    
    pearson_ratings = list()
    for user_idx in range(0, users_mat.size-1):
        
        user_y = users_mat[user_idx]

        # get the cut set of rated movies of both users.
        intersec_movies = custom_intersection(target_user, user_y)

        # compute pearson correlation of these users
        if(len(intersec_movies)>10): # if the intersection is too small, we cant apply statistics to it
            ratings_x = np.array(intersec_movies)
            pear = np.corrcoef(ratings_x[:,0],ratings_x[:,1])
            pearson_ratings.append([user_idx, pear[0,1]])
            

    pearson_ratings.sort(key=lambda tup: tup[1], reverse = True)
    
    return (pearson_ratings[0:k-1])# (list of tuples (user, pearson_corr))




In [7]:
# Divide Data in training and test batch
train = user_ratings[0:2000000]
test =  user_ratings[2000001:2000300]
nn_per_user = list()

In [86]:
#load training savegame
import pickle
with open("nn_per_user.txt", "rb") as fp:   # Unpickling
    nn_per_user = pickle.load(fp)


In [87]:
print(len(nn_per_user))

299


In [88]:
# perform training with k_nn
k = 10;
min_intersection = 10;

for idx in range (np.max(len(nn_per_user)-1, 0),test.size-1):
    user = test[idx]
    nn = k_nn_memorized(idx, k, train, user, min_intersection)
    nn_per_user.append(nn)
    
    b = "Trained on user " + str(idx) + " of " + str(test.size-1) + ". In percent: " + str(idx/(test.size-1) * 100)
    print (b, end="\r")
        


In [85]:
# safe the training:
import pickle
with open("nn_per_user.txt", "wb") as fp:   #Pickling
    pickle.dump(nn_per_user, fp)
 

In [63]:
# get results
realVsPred = list()
for idx in range (0,test.size): #test.size-1
    user = test[idx]
    for j_user in range (0,len(user)-1):
        idx_movie = user[j_user][0]
        nn = nn_per_user[idx]
        
        if(len(nn)!= 0):
            #print(nn)
            ratings = list();
            
            for idy in range(0,len(nn)):
                rating_y = getRating(nn[idy][0], idx_movie)
                
                if(rating_y != -1):
                    ratings.append(rating_y)

            if(len(ratings) != 0):
                prediction = np.mean(ratings)
                print("Test user: ", idx, " Movie ", idx_movie, " , prediction (", len(ratings), "neighbor ratings available): " , prediction, " real ", user[j_user][1])
                realVsPred.append((prediction, user[j_user][1]))
            #else: 
                #print("No feasible ratings for user ", idx , " and  movie ", idx_movie ,".")

Test user:  4  Movie  1307  , prediction ( 1 neighbor ratings available):  4.0  real  2
Test user:  4  Movie  4432  , prediction ( 2 neighbor ratings available):  4.0  real  4
Test user:  4  Movie  7617  , prediction ( 1 neighbor ratings available):  5.0  real  5
Test user:  4  Movie  12232  , prediction ( 2 neighbor ratings available):  2.5  real  5
Test user:  4  Movie  13050  , prediction ( 1 neighbor ratings available):  3.0  real  3
Test user:  4  Movie  14103  , prediction ( 2 neighbor ratings available):  4.5  real  5
Test user:  4  Movie  15394  , prediction ( 2 neighbor ratings available):  3.5  real  5
Test user:  66  Movie  30  , prediction ( 2 neighbor ratings available):  4.0  real  3
Test user:  66  Movie  457  , prediction ( 2 neighbor ratings available):  4.0  real  4
Test user:  66  Movie  468  , prediction ( 1 neighbor ratings available):  3.0  real  2
Test user:  66  Movie  571  , prediction ( 1 neighbor ratings available):  5.0  real  4
Test user:  66  Movie  1220  

In [65]:
# evaluate results
RMSE = 0
for pair in realVsPred:
    RMSE += (pair[0]-pair[1])*(pair[0]-pair[1])
RMSE = np.sqrt(RMSE)
RMSE /= len(realVsPred)
print("RMSE of the k_nn predictions is :", RMSE)

RMSE of the k_nn predictions is : 0.035873915497458296


## Discussion ##
Not bad, given that we have very few neighbors that have watched the same movie!
But there some things to note:
* We have only calculated the RMSE, if we have neighbors available, that have actually rated that move. (Huge Bias in the results!)
* Quite often,  we have only a few neighbors that have watched the same move.

Both problems are partially caused by the same underlying problem.
We have calculated the $k$ nearest neighbors for a given user $x$. This does not factor in, that the $k$ nearest neighbors must have watched the movie. 

We can overcome this problem in two ways.
1. We can compute live for a given movie and a given user the $k$ nearest neighbors and use their rating (computationally quite expensive, since we have to perform the training above for each movie).
2. We store not only the $k$ neighbors with the best pearson coefficient, but all users with descending pearson coefficient. This is quite memory expensive, given the amount of users available.

We additionally go for a weaker version of approach 2 by storing not only the $k$ nearest, but the $N*k$ nearest neighbors, where $N$ is a scaling factor.